In [49]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
df = spark.read.format("csv")\
    .option("quote","\"")\
    .option("escape","\"")\
    .option("multiline",True)\
    .option("delimeter",",")\
    .option("pathGlobFilter", "{*.csv}")\
    .option("recursiveFileLookup", "true")\
    .load("Files/Tables/SupplyChain/ProductionControl/Transaction/ProdCalcTrans")
# df now is a Spark DataFrame containing CSV data from "Files/Tables/SupplyChain/ProductionControl/Transaction/ProdCalcTrans". All csv files

StatementMeta(, 5e2986c2-ff7a-4005-89fb-d6780e685636, 51, Finished, Available, Finished)

In [50]:
df = df.sort(df['_c5'].desc())

StatementMeta(, 5e2986c2-ff7a-4005-89fb-d6780e685636, 52, Finished, Available, Finished)

In [36]:
testdf = df.filter(df._c5 == 'PD-1233322')
display(testdf)

StatementMeta(, 5e2986c2-ff7a-4005-89fb-d6780e685636, 38, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 41078456-d0db-4c80-a719-f7001edf9914)

In [51]:
renamed_df = df.select(
                col("_c5").alias("Production"),\
                col("_c6").alias("Level"),\
                col("_c8").alias("Type"),\
                #Type Legend (0 = Production, 1 = Item, 2 = BOM, 3 = SubContract, 4 = Setup, 5 = Process) 
                col("_c11").alias("SubProduction"),\
                col("_c13").alias("CostGroup"),\
                col("_c15").alias("EstimatedProcessCost"),\
                col("_c16").alias("EstimatedSetupCost"),\
                col("_c19").alias("Date"),\
                col("_c20").alias("ItemResource"),\
                #col("_c24").alias("ConsumptionPerUnit"),\
                col("_c28").alias("RealizedCostAmount"),\
                col("_c32").alias("OperationNo"),\
                col("_c34").alias("Number")
                )

StatementMeta(, 5e2986c2-ff7a-4005-89fb-d6780e685636, 53, Finished, Available, Finished)

In [52]:
renamed_df = renamed_df.withColumn("Date",col("Date").cast(DateType()))

StatementMeta(, 5e2986c2-ff7a-4005-89fb-d6780e685636, 54, Finished, Available, Finished)

In [53]:
renamed_df = renamed_df.withColumn('Type', regexp_replace('Type', '0','Production'))
renamed_df = renamed_df.withColumn('Type', regexp_replace('Type', '1','Item'))
renamed_df = renamed_df.withColumn('Type', regexp_replace('Type', '2','Process'))
renamed_df = renamed_df.withColumn('Type', regexp_replace('Type', '3','SubContract'))
renamed_df = renamed_df.withColumn('Type', regexp_replace('Type', '4','Setup'))
renamed_df = renamed_df.withColumn('Type', regexp_replace('Type', '5','Process'))

StatementMeta(, 5e2986c2-ff7a-4005-89fb-d6780e685636, 55, Finished, Available, Finished)

In [54]:
renamed_df = renamed_df.withColumn('EstimatedProcessCost', round(renamed_df['EstimatedProcessCost'],2))
renamed_df = renamed_df.withColumn('EstimatedSetupCost', round(renamed_df['EstimatedSetupCost'],2))
renamed_df = renamed_df.withColumn('RealizedCostAmount', round(renamed_df['RealizedCostAmount'],2))

StatementMeta(, 5e2986c2-ff7a-4005-89fb-d6780e685636, 56, Finished, Available, Finished)

In [55]:
#display(renamed_df)

StatementMeta(, 5e2986c2-ff7a-4005-89fb-d6780e685636, 57, Finished, Available, Finished)

In [56]:
delta_table_path = "Tables/ProdCostingTable" #fill in your delta table path 
renamed_df.write.format("delta").mode("overwrite").option("overwriteSchema","true").save(delta_table_path)

StatementMeta(, 5e2986c2-ff7a-4005-89fb-d6780e685636, 58, Finished, Available, Finished)